# Week 6: Computer Vision I — CNN Fundamentals
## COMP 9130 — Applied Artificial Intelligence

---

**Today's Dataset:** Wildfire Detection Image Data (~1,900 images, binary: fire vs no-fire)

**Learning Objectives:**
1. Build a CNN from scratch using Conv2D and pooling layers
2. Train a CNN for binary image classification (wildfire detection)
3. Use Keras augmentation layers to reduce overfitting
4. Visualize CNN internals — feature maps and learned filters

**Today's Tasks:**
| Time | Task | Points |
|------|------|--------|
| 0:00–0:20 | Quiz 5 | 5 pts |
| 0:30–1:20 | Task 1: Build & Train a Basic CNN | 5 pts |
| 1:30–2:20 | Task 2: Data Augmentation | 5 pts |
| 2:20–2:55 | Task 3: Visualize CNN Internals | 5 pts |

**Pair Programming Reminder:** One person types, one person thinks. Switch roles between tasks!

---

## Setup & Imports

Run this cell first. Make sure you have GPU enabled: **Runtime → Change runtime type → T4 GPU**

In [1]:
# ============================================================
# SETUP — Run this first!
# ============================================================
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import os
import random

# Verify versions
print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version:      {np.__version__}")

# Check GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"\n✅ GPU available: {gpus[0].name}")
else:
    print("\n⚠️  No GPU detected. Go to Runtime → Change runtime type → T4 GPU")

# Reproducibility
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

ModuleNotFoundError: No module named 'tensorflow'

## Download & Load the Wildfire Dataset

We're using a real wildfire detection dataset from Kaggle: ~1,900 images of fire/no-fire scenes.

In [ ]:
# Download the dataset
!pip install -q kagglehub
import kagglehub

dataset_path = kagglehub.dataset_download("brsdincer/wildfire-detection-image-data")
print(f"Dataset downloaded to: {dataset_path}")

# Print structure to understand folder layout
print("\nDataset structure:")
for root, dirs, files in os.walk(dataset_path):
    level = root.replace(dataset_path, '').count(os.sep)
    indent = '  ' * level
    print(f"{indent}{os.path.basename(root)}/  ({len(files)} files)")
    if level >= 3:
        continue

# Find the directory with exactly 2 class subfolders containing images
data_dir = None
for root, dirs, files in os.walk(dataset_path):
    dirs[:] = [d for d in dirs if not d.startswith('.')]
    if len(dirs) == 2:
        has_images = False
        for d in dirs:
            subpath = os.path.join(root, d)
            img_files = [f for f in os.listdir(subpath)
                        if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            if len(img_files) > 10:
                has_images = True
                break
        if has_images:
            data_dir = root
            break

if data_dir is None:
    raise FileNotFoundError("Could not find class folders. Check the structure above.")

print(f"\n✅ Data directory: {data_dir}")
for d in sorted(os.listdir(data_dir)):
    sub = os.path.join(data_dir, d)
    if os.path.isdir(sub):
        n = len([f for f in os.listdir(sub) if f.lower().endswith(('.png','.jpg','.jpeg'))])
        print(f"   - {d}: {n} images")

In [ ]:
# Load images into TensorFlow datasets
IMG_SIZE = 250  # Original resolution (250x250)
BATCH_SIZE = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=42,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=42,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

class_names = train_ds.class_names
print(f"Class names: {class_names}")

## Explore the Data

Let's see what our images look like and understand their properties.

In [ ]:
# Count images per class
for class_name in class_names:
    class_path = os.path.join(data_dir, class_name)
    n_images = len([f for f in os.listdir(class_path)
                    if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    print(f"  {class_name}: {n_images} images")

In [ ]:
# Visualize sample images from both classes
plt.figure(figsize=(14, 8))

# Keras assigns classes alphabetically: fire=0, nofire=1
fire_class = class_names.index('fire') if 'fire' in class_names else 0
nofire_class = 1 - fire_class

for images, labels in train_ds.take(1):
    fire_mask = labels.numpy().flatten() == fire_class
    nofire_mask = labels.numpy().flatten() == nofire_class

    # Show fire images (top row)
    fire_images = images[fire_mask]
    for i in range(min(5, len(fire_images))):
        plt.subplot(2, 5, i + 1)
        plt.imshow(fire_images[i].numpy().astype('uint8'))
        plt.title('Fire', fontsize=11, fontweight='bold', color='red')
        plt.axis('off')

    # Show no-fire images (bottom row)
    nofire_images = images[nofire_mask]
    for i in range(min(5, len(nofire_images))):
        plt.subplot(2, 5, i + 6)
        plt.imshow(nofire_images[i].numpy().astype('uint8'))
        plt.title('No Fire', fontsize=11, fontweight='bold', color='green')
        plt.axis('off')

plt.suptitle('Wildfire Detection Dataset — Sample Images', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Inspect image properties
for images, labels in train_ds.take(1):
    print(f"Batch shape:  {images.shape}")
    print(f"Label shape:  {labels.shape}")
    print(f"Pixel range:  [{images.numpy().min():.0f}, {images.numpy().max():.0f}]")
    print(f"\nSingle image shape: {images[0].shape}")
    print(f"  → Height: {images[0].shape[0]}, Width: {images[0].shape[1]}, Channels: {images[0].shape[2]}")

# Why CNNs?
print(f"\n💡 If we flattened this image: {250*250*3:,} input features!")
print(f"   Compare to MNIST flattened: {28*28} input features")
print(f"   → This is why CNNs use weight sharing through convolutions!")

In [ ]:
# Optimize data pipeline
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
print("✅ Data pipeline optimized")

In [ ]:
# ============================================================
# PROVIDED: Helper function for plotting training curves
# (You'll use this in Tasks 1 and 2)
# ============================================================

def plot_training_curves(history, title='Model Training'):
    """Plot accuracy and loss curves for training and validation."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    ax1.plot(history.history['accuracy'], label='Train', linewidth=2)
    ax1.plot(history.history['val_accuracy'], label='Validation', linewidth=2)
    ax1.set_title(f'{title} — Accuracy', fontweight='bold')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim([0.4, 1.05])

    ax2.plot(history.history['loss'], label='Train', linewidth=2)
    ax2.plot(history.history['val_loss'], label='Validation', linewidth=2)
    ax2.set_title(f'{title} — Loss', fontweight='bold')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    final_train_acc = history.history['accuracy'][-1]
    final_val_acc = history.history['val_accuracy'][-1]
    gap = final_train_acc - final_val_acc
    print(f"\n📊 Final Results:")
    print(f"   Train Accuracy:      {final_train_acc:.4f} ({final_train_acc*100:.1f}%)")
    print(f"   Validation Accuracy: {final_val_acc:.4f} ({final_val_acc*100:.1f}%)")
    print(f"   Gap (overfit check): {gap:.4f} ({gap*100:.1f}%)")
    if gap > 0.15:
        print(f"   ⚠️  Large gap — model is OVERFITTING")
    elif gap > 0.05:
        print(f"   ⚡ Moderate gap — some overfitting")
    else:
        print(f"   ✅ Small gap — good generalization!")


def show_predictions(model, dataset, class_names, n=10):
    """Show model predictions on sample images."""
    plt.figure(figsize=(16, 4))
    for images, labels in dataset.take(1):
        predictions = model.predict(images, verbose=0)
        for i in range(min(n, len(images))):
            plt.subplot(2, 5, i + 1)
            plt.imshow(images[i].numpy().astype('uint8'))
            pred_prob = predictions[i][0]
            pred_label = 1 if pred_prob > 0.5 else 0
            true_label = int(labels[i].numpy())
            color = 'green' if pred_label == true_label else 'red'
            plt.title(f'Pred: {class_names[pred_label]}\n({pred_prob:.2f})\nTrue: {class_names[true_label]}',
                     fontsize=9, color=color, fontweight='bold')
            plt.axis('off')
    plt.suptitle('Predictions (Green=Correct, Red=Wrong)', fontweight='bold')
    plt.tight_layout()
    plt.show()

print("✅ Helper functions loaded")

---

# ═══════════════════════════════════════════
# TASK 1: Build & Train a Basic CNN (50 min)
# ═══════════════════════════════════════════

Build your first Convolutional Neural Network for wildfire detection!

**Architecture to build:**
```
Input (250×250×3) → Rescaling (0-1)
  → Conv2D(32, 3×3, relu) → MaxPool(2×2)
  → Conv2D(64, 3×3, relu) → MaxPool(2×2)
  → Conv2D(128, 3×3, relu) → MaxPool(2×2)
  → Flatten → Dense(128, relu) → Dense(1, sigmoid)
```

**Key concepts:**
- `Rescaling(1./255)`: Normalizes pixels from [0, 255] to [0, 1]
- `Conv2D(filters, kernel_size, activation)`: Applies learned filters to detect patterns
- `MaxPooling2D(pool_size)`: Reduces spatial dimensions by keeping max values
- `Flatten`: Converts 3D feature maps to 1D for the Dense layers
- `Dense(1, sigmoid)`: Outputs probability of fire (binary classification)

In [ ]:
# ============================================================
# TODO: Build your CNN model
# ============================================================
# Follow the architecture diagram above.
# Hint: Use models.Sequential([...]) with the layers listed.

model_basic = models.Sequential([
    # TODO: Add Rescaling layer (normalize pixels to 0-1)
    #       Don't forget input_shape=(IMG_SIZE, IMG_SIZE, 3)

    # TODO: Conv Block 1 — Conv2D(32 filters, 3x3, relu) + MaxPooling2D(2x2)

    # TODO: Conv Block 2 — Conv2D(64 filters, 3x3, relu) + MaxPooling2D(2x2)

    # TODO: Conv Block 3 — Conv2D(128 filters, 3x3, relu) + MaxPooling2D(2x2)

    # TODO: Classification head — Flatten, Dense(128, relu), Dense(1, sigmoid)

])

# Display the architecture
model_basic.summary()

In [ ]:
# ============================================================
# TODO: Compile the model
# ============================================================
# Use:
#   optimizer = 'adam'
#   loss = 'binary_crossentropy' (because binary classification)
#   metrics = ['accuracy']

# YOUR CODE HERE


In [ ]:
# ============================================================
# TODO: Train the model for 50 epochs
# ============================================================
# Use model_basic.fit() with:
#   - train_ds as training data
#   - val_ds as validation_data
#   - epochs=15
# Save the result to history_basic

EPOCHS = 50

# YOUR CODE HERE
# history_basic = ...


In [ ]:
# Plot training curves (use the provided helper function)
plot_training_curves(history_basic, title='Basic CNN (No Augmentation)')

In [ ]:
# Show predictions on validation images
show_predictions(model_basic, val_ds, class_names)

## ✅ Task 1 Checkpoint — Show your instructor

| Criteria | Points |
|----------|--------|
| CNN built, trained, ≥75% val accuracy, analysis answered | 5 pts |
| CNN trains but accuracy <75% or incomplete analysis | 3 pts |
| Attempted but doesn't train properly | 1 pt |

### 📝 Task 1 Analysis Questions

**Answer using YOUR specific results (numbers from YOUR training).**

**Q1:** What is your model's final training accuracy and validation accuracy? Is the model overfitting? How can you tell from the training curves?

*Your answer:*


**Q2:** Look at the `model.summary()` output. Which layer has the most parameters? Why does the Flatten → Dense connection create so many parameters?

*Your answer:*


**Q3:** Why do we increase the number of filters (32 → 64 → 128) as we go deeper? What happens to the spatial dimensions at the same time?

*Your answer:*


---

# ══════════════════════════════════════════════
# TASK 2: Data Augmentation (50 min)
# ══════════════════════════════════════════════

Your Task 1 model likely overfits (training accuracy >> validation accuracy). Let's fix that with **data augmentation** — applying random transforms to training images so the model sees different versions each epoch.

We'll use modern **Keras augmentation layers** that integrate directly into the model.

## Part 2A: Visualize What Augmentation Does

In [ ]:
# ============================================================
# Define augmentation layers
# ============================================================
# These layers apply random transformations during training ONLY.
# During prediction, they automatically deactivate.

data_augmentation = models.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
], name='data_augmentation')

print("Augmentation layers:")
for layer in data_augmentation.layers:
    print(f"  - {layer.__class__.__name__}")

In [ ]:
# Visualize augmentation on a single image
plt.figure(figsize=(12, 12))

for images, labels in train_ds.take(1):
    sample_image = images[0]

    plt.subplot(4, 4, 1)
    plt.imshow(sample_image.numpy().astype('uint8'))
    plt.title('ORIGINAL', fontweight='bold', fontsize=11)
    plt.axis('off')

    for i in range(15):
        augmented = data_augmentation(tf.expand_dims(sample_image, 0), training=True)
        plt.subplot(4, 4, i + 2)
        plt.imshow(augmented[0].numpy().astype('uint8'))
        plt.title(f'Augmented #{i+1}', fontsize=9)
        plt.axis('off')

plt.suptitle('Data Augmentation — Same Image, Different Transforms', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("💡 Each epoch, the model sees a DIFFERENT version of every training image.")

## Part 2B: Build CNN with Augmentation + Dropout

In [ ]:
# ============================================================
# TODO: Build a CNN with augmentation layers built in
# ============================================================
# Same CNN architecture as Task 1, BUT:
#   1. Add augmentation layers AFTER Rescaling (before Conv layers)
#   2. Add Dropout(0.3) AFTER Flatten (before Dense)
#   3. #
# Architecture:
#   Rescaling → RandomFlip → RandomRotation → RandomZoom → RandomContrast
#   → Conv2D(32) → MaxPool → Conv2D(64) → MaxPool → Conv2D(128) → MaxPool
#   → Flatten → Dropout(0.3) → Dense(128, relu) → Dense(1, sigmoid)

model_augmented = models.Sequential([
    # TODO: Rescaling layer with input_shape

    # TODO: Add the 4 augmentation layers

    # TODO: Same 3 Conv+Pool blocks as Task 1

    # TODO: Flatten → Dropout(0.3) → Dense(128, relu) → Dense(1, sigmoid)

])

model_augmented.summary()

In [ ]:
# ============================================================
# TODO: Compile and train the augmented model
# ============================================================
# Same compile settings as Task 1
# Train for 50 epochs (same as basic) (same as basic for fair comparison)

EPOCHS_AUG = 50

# YOUR CODE HERE
# model_augmented.compile(...)
# history_augmented = model_augmented.fit(...)


## Part 2C: Compare Results

In [ ]:
# ============================================================
# TODO: Plot both models' training curves side by side
# ============================================================
# Create a figure with 2 subplots (1 row, 2 columns)
# Left: Basic CNN accuracy curves (train + val)
# Right: Augmented CNN accuracy curves (train + val)
#
# Hint: Use history_basic.history['accuracy'] and
#       history_augmented.history['accuracy']

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# YOUR CODE HERE — plot train/val accuracy for both models


plt.suptitle('Effect of Data Augmentation on Overfitting', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# TODO: Create a comparison table
# ============================================================
# Fill in your actual numbers from training

import pandas as pd

basic_train = history_basic.history['accuracy'][-1]
basic_val = history_basic.history['val_accuracy'][-1]
aug_train = history_augmented.history['accuracy'][-1]
aug_val = history_augmented.history['val_accuracy'][-1]

comparison = pd.DataFrame({
    'Model': ['Basic CNN', 'Augmented CNN'],
    'Train Acc': [f'{basic_train:.4f}', f'{aug_train:.4f}'],
    'Val Acc': [f'{basic_val:.4f}', f'{aug_val:.4f}'],
    'Gap': [f'{basic_train - basic_val:.4f}', f'{aug_train - aug_val:.4f}']
})

print("\n" + "="*60)
print("MODEL COMPARISON")
print("="*60)
print(comparison.to_string(index=False))
print("="*60)

In [ ]:
# Show predictions from augmented model
show_predictions(model_augmented, val_ds, class_names)

## ✅ Task 2 Checkpoint — Show your instructor

| Criteria | Points |
|----------|--------|
| Augmented model trained, comparison table filled, analysis answered | 5 pts |
| Model trains but incomplete comparison | 3 pts |
| Attempted but augmentation not working | 1 pt |

### 📝 Task 2 Analysis Questions

**Answer using YOUR specific comparison numbers.**

**Q1:** Compare the train-validation gap between your basic CNN and augmented CNN. Which model overfits more? Use specific numbers from your comparison table.

*Your answer:*


**Q2:** Why does the augmented model have LOWER training accuracy than the basic model? Is that a problem?

*Your answer:*


**Q3:** We added Dropout layers in addition to augmentation. Why both? What does each technique address?

*Your answer:*


---

# ════════════════════════════════════════════════
# TASK 3: Visualize CNN Internals (35 min)
# ════════════════════════════════════════════════

Now let's look INSIDE the CNN to understand what it actually learned.

We'll visualize:
- **Feature maps** (activations): What the CNN "sees" at each layer
- **Learned filters** (weights): The patterns the CNN learned to detect

## Part 3A: Visualize Feature Maps

In [ ]:
# ============================================================
# TODO: Create a sub-model that outputs feature maps
# ============================================================
# We need to extract the outputs of each Conv2D layer.
# In Keras 3.x, we rebuild the forward pass with a Functional API Input.
#
# Steps:
#   1. Create keras.Input with shape (IMG_SIZE, IMG_SIZE, 3)
#   2. Loop through model_basic.layers, passing x through each
#   3. Collect outputs from Conv2D layers
#   4. Build models.Model(inputs=inp, outputs=conv_outputs)

inp = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

x = inp
conv_outputs = []
conv_names = []
for layer in model_basic.layers:
    x = layer(x)
    if isinstance(layer, layers.Conv2D):
        conv_outputs.append(x)
        conv_names.append(layer.name)
        print(f"  {layer.name}: output shape = {x.shape}")

# TODO: Build the feature map model using models.Model()
# YOUR CODE HERE
# feature_map_model = ...


In [ ]:
# Select a fire image and a no-fire image for comparison
# Keras assigns classes alphabetically: fire=0, nofire=1
print(f"Class mapping: {class_names}")

fire_class = class_names.index('fire') if 'fire' in class_names else 0
nofire_class = 1 - fire_class

for images, labels_batch in val_ds.take(1):
    fire_idx = None
    nofire_idx = None
    for i in range(len(labels_batch)):
        label = int(labels_batch[i].numpy())
        if label == fire_class and fire_idx is None:
            fire_idx = i
        elif label == nofire_class and nofire_idx is None:
            nofire_idx = i
        if fire_idx is not None and nofire_idx is not None:
            break

    test_fire = images[fire_idx]
    test_nofire = images[nofire_idx]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))
ax1.imshow(test_fire.numpy().astype('uint8'))
ax1.set_title('Test: Fire', fontweight='bold')
ax1.axis('off')
ax2.imshow(test_nofire.numpy().astype('uint8'))
ax2.set_title('Test: No Fire', fontweight='bold')
ax2.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# TODO: Get feature maps for the fire image and display them
# ============================================================
# Steps:
#   1. Add batch dimension: tf.expand_dims(test_fire, 0)
#   2. Pass through feature_map_model.predict()
#   3. For each layer, show the first 16 feature maps in a 2x8 grid
#
# Hint: feature_maps[layer_idx] has shape (1, H, W, num_filters)
#       Display feature_maps[layer_idx][0, :, :, filter_idx]

test_input = tf.expand_dims(test_fire, 0)
feature_maps = feature_map_model.predict(test_input, verbose=0)

# conv_names was defined in the cell above

# YOUR CODE HERE — display feature maps for each layer
# Use plt.subplot() and plt.imshow() with cmap='viridis'


## Part 3B: Visualize Learned Filters

In [ ]:
# ============================================================
# TODO: Visualize learned filters from each Conv layer
# ============================================================
# Steps:
#   1. Loop through Conv2D layers in model_basic
#   2. Get weights with layer.get_weights() → returns [filters, biases]
#   3. Normalize filter values to [0, 1] for display
#   4. Show first 16 filters in a 2x8 grid
#
# For the first layer (3 input channels), you can display filters as RGB
# For deeper layers, show the mean across input channels

for layer in model_basic.layers:
    if isinstance(layer, layers.Conv2D):
        filters, biases = layer.get_weights()
        print(f"\nLayer: {layer.name}, Filter shape: {filters.shape}")

        # YOUR CODE HERE — normalize and display filters


## Part 3C: Activation Heatmap

In [ ]:
# ============================================================
# TODO: Create an activation heatmap
# ============================================================
# This shows WHERE the CNN focuses in the image.
# Steps:
#   1. Get feature maps from the LAST Conv layer for fire image
#   2. Average across all filter channels → (H, W) heatmap
#   3. Normalize to [0, 1]
#   4. Resize to image size using tf.image.resize()
#   5. Plot: original, heatmap, and overlay (original + heatmap with alpha=0.4)

# Get feature maps for both images
fmaps_fire = feature_map_model.predict(tf.expand_dims(test_fire, 0), verbose=0)
fmaps_nofire = feature_map_model.predict(tf.expand_dims(test_nofire, 0), verbose=0)

# Get last layer's feature maps
fmaps_last = fmaps_fire[-1]  # shape: (1, H, W, 128)

# YOUR CODE HERE
# heatmap = np.mean(fmaps_last[0], axis=-1)  # Average across filters
# Normalize, resize, and plot


## ✅ Task 3 Checkpoint — Show your instructor

| Criteria | Points |
|----------|--------|
| Feature maps + filters visualized, heatmap shown, analysis answered | 5 pts |
| One visualization completed with partial analysis | 3 pts |
| Attempted but visualization not working | 1 pt |

### 📝 Task 3 Analysis Questions

**Q1:** Look at the feature maps from Layer 1 vs Layer 3. How do they differ? What does this tell you about hierarchical feature learning?

*Your answer:*


**Q2:** Compare the activation heatmap for the fire image vs the no-fire image. Does the CNN focus on different regions? Does this make sense?

*Your answer:*


**Q3:** Look at the learned filters from the first Conv layer. Can you identify any edge detectors or color detectors?

*Your answer:*


---

# 🎁 BONUS: Grad-CAM (Optional — for early finishers)

Grad-CAM uses gradients to weight each filter's importance, creating a more principled heatmap.

In [ ]:
# ============================================================
# BONUS: Grad-CAM implementation
# ============================================================

def make_gradcam_heatmap(img_tensor, model, last_conv_layer_name):
    """Generate Grad-CAM heatmap for a given image and model."""
    # Rebuild as Functional model (Keras 3.x compatible)
    inp = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = inp
    conv_output = None
    for layer in model.layers:
        x = layer(x)
        if layer.name == last_conv_layer_name:
            conv_output = x
    grad_model = models.Model(inputs=inp, outputs=[conv_output, x])

    # Compute gradients
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_tensor)
        predicted_class = predictions[0]

    grads = tape.gradient(predicted_class, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / (tf.math.reduce_max(heatmap) + 1e-8)
    return heatmap.numpy()

# Get the name of the last Conv layer
last_conv_name = [l.name for l in model_basic.layers if isinstance(l, layers.Conv2D)][-1]
print(f"Last Conv layer: {last_conv_name}")

# Generate Grad-CAM for fire image
heatmap = make_gradcam_heatmap(
    tf.expand_dims(test_fire, 0),
    model_basic,
    last_conv_name
)

# Resize and overlay
heatmap_resized = tf.image.resize(heatmap[..., np.newaxis], (IMG_SIZE, IMG_SIZE)).numpy()[:, :, 0]

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(14, 4))
ax1.imshow(test_fire.numpy().astype('uint8'))
ax1.set_title('Original', fontweight='bold')
ax1.axis('off')

ax2.imshow(heatmap_resized, cmap='jet')
ax2.set_title('Grad-CAM Heatmap', fontweight='bold')
ax2.axis('off')

ax3.imshow(test_fire.numpy().astype('uint8'))
ax3.imshow(heatmap_resized, cmap='jet', alpha=0.4)
ax3.set_title('Grad-CAM Overlay', fontweight='bold')
ax3.axis('off')

plt.suptitle('Grad-CAM: Where the CNN looks to decide "Fire"', fontweight='bold')
plt.tight_layout()
plt.show()

print("Grad-CAM uses gradients to weight which filters matter most.")
print("This is more principled than simply averaging all feature maps.")

---

# 🎓 Session Complete!

**Key Takeaways:**
1. CNNs preserve spatial structure — unlike MLPs that flatten everything
2. Convolution + pooling = efficient feature extraction with fewer parameters
3. Data augmentation reduces overfitting by creating virtual training variety
4. CNNs learn hierarchically: edges → textures → objects
5. Visualization builds understanding and trust in what the model learns

**Next Week:** Transfer Learning — use models pre-trained on 1.2 MILLION ImageNet images!

**Mini Project 5** is now posted on D2L. Due: Week 7.

---